# nnU-Net v2 on BraTS 2024 Subset (Colab Ready)

## Cell 1 — GPU Check

In [ ]:

import torch
print("CUDA available:", torch.cuda.is_available())
assert torch.cuda.is_available(), "Enable GPU in Runtime → Change runtime type"
print("GPU:", torch.cuda.get_device_name(0))


## Cell 2 — Clone Repository

In [ ]:

!git clone https://github.com/<YOUR_USERNAME>/iuwo-segmentation.git
%cd iuwo-segmentation


## Cell 3 — Install Dependencies

In [ ]:

!pip install --upgrade pip
!pip install nnunetv2 nibabel


## Cell 4 — nnU-Net Environment Variables

In [ ]:

import os
os.environ["nnUNet_raw"] = "/content/nnUNet_raw"
os.environ["nnUNet_preprocessed"] = "/content/nnUNet_preprocessed"
os.environ["nnUNet_results"] = "/content/nnUNet_results"

!mkdir -p $nnUNet_raw $nnUNet_preprocessed $nnUNet_results


## Cell 5 — Upload brats_subset_clean.zip

Use Colab file uploader

## Cell 6 — Extract Dataset

In [ ]:

!mkdir -p $nnUNet_raw/Dataset001_BraTS
!unzip brats_subset_clean.zip -d $nnUNet_raw/Dataset001_BraTS


## Cell 7 — Convert to nnU-Net Structure

In [ ]:

import shutil, pathlib

base = pathlib.Path("/content/nnUNet_raw/Dataset001_BraTS")
imagesTr = base / "imagesTr"
labelsTr = base / "labelsTr"
imagesTs = base / "imagesTs"

imagesTr.mkdir(exist_ok=True)
labelsTr.mkdir(exist_ok=True)
imagesTs.mkdir(exist_ok=True)

EVAL = {"BraTS-GLI-00008-103","BraTS-GLI-00009-100","BraTS-GLI-00085-100"}

for p in base.iterdir():
    if not p.is_dir() or p.name in {"imagesTr","labelsTr","imagesTs"}:
        continue
    flair = list(p.glob("*flair*.nii*"))[0]
    seg   = list(p.glob("*seg*.nii*"))[0]
    if p.name in EVAL:
        shutil.copy(flair, imagesTs / f"{p.name}_0000.nii.gz")
    else:
        shutil.copy(flair, imagesTr / f"{p.name}_0000.nii.gz")
        shutil.copy(seg, labelsTr / f"{p.name}.nii.gz")


## Cell 8 — Convert Labels to Binary

In [ ]:

import nibabel as nib, numpy as np
from pathlib import Path

labels_dir = Path("/content/nnUNet_raw/Dataset001_BraTS/labelsTr")
for f in labels_dir.glob("*.nii.gz"):
    img = nib.load(f)
    data = img.get_fdata()
    bin_mask = (data > 0).astype(np.uint8)
    nib.save(nib.Nifti1Image(bin_mask, img.affine, img.header), f)


## Cell 9 — dataset.json

In [ ]:

import json, os

dataset = {
    "channel_names": {"0": "FLAIR"},
    "labels": {"background": 0, "tumor": 1},
    "numTraining": len(os.listdir("/content/nnUNet_raw/Dataset001_BraTS/imagesTr")),
    "file_ending": ".nii.gz"
}

with open("/content/nnUNet_raw/Dataset001_BraTS/dataset.json","w") as f:
    json.dump(dataset,f,indent=2)


## Cell 10 — Planning & Preprocessing

In [ ]:

!nnUNetv2_plan_and_preprocess -d 1 --verify_dataset_integrity


## Cell 11 — Training

In [ ]:

!nnUNetv2_train 1 3d_fullres 0 --device cuda


## Cell 12 — Inference

In [ ]:

!nnUNetv2_predict -i $nnUNet_raw/Dataset001_BraTS/imagesTs -o predictions_nnunet -d 1 -c 3d_fullres -f 0 --device cuda
